## Imports

In [39]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm
from sklearn.ensemble import IsolationForest
import plotly.express as px
import numpy as np
import warnings
warnings.filterwarnings("ignore")


## Funció Alineament de Dades

In [22]:
def alignSeries(serie, lag):
    minutesRow = 30
    posToMove = int(lag / minutesRow)  
    outputSerie = serie['data'].to_numpy() 
    outputSerie = np.pad(outputSerie, (0,posToMove), 'constant', constant_values=(0))
    return outputSerie

## Carregar Dades

In [11]:
def calcularAnomaly(df, col, cont=0.005):
    #Crear una versio horaria del dataset
    df['timestamp'] = pd.to_datetime(df.index)
    df = df.set_index('timestamp').resample("H").mean().reset_index()
    
    #Usem el metode de Isolation Forest per trobar anomalies a les precipitacions
    output = df[[col, 'timestamp']]
    temp = df[[col, 'timestamp']]
    model=IsolationForest(n_estimators=50, max_samples='auto', contamination=cont,max_features=1.0)
    model.fit(temp[[col]].values)
    output['outliers']=pd.Series(model.predict(temp[[col]].values)).apply(lambda x: 'yes' if (x == -1) else 'no' )
    output.rename(columns={col:'data'}, inplace=True)
    
    return output

In [12]:
# Carregar dades pluja
def loadDataPrecipitacions():
    precipitacions = pd.read_excel("finalsDF/DF_SMC.xlsx", index_col=0)
    return precipitacions

In [13]:
# Carregar dades Alt Ter
def loadDataAltTer(nom, altTer):
    
    altTer = calcularAnomaly(altTer, nom)
    
    return altTer

In [14]:
# Carregar dades Baix Ter
def loadDataBaixTer(nom, baixTer):
    
    baixTer = calcularAnomaly(baixTer, nom)

    return baixTer

In [15]:
#Carregar dades precipitacions Alt Ter:
def loadDataPrecipitacions(nom, precipitacions):
    
    #Carregar dades
    precipitacions = calcularAnomaly(precipitacions, nom)
    
    #Moving Average
    precipitacions['data'] = precipitacions['data'].rolling(window=5).mean()
    precipitacions['data'] = precipitacions['data'].rolling(window=5).mean()
    
    return precipitacions

In [16]:
altTer = pd.read_excel("Dataframes/df_imputedAltTerKNN.xlsx", index_col=0)
baixTer = pd.read_excel("Dataframes/df_imputedBaixTerKNN.xlsx", index_col=0)
precipitacions = pd.read_csv("finalsDF/DF_SMC.csv", index_col=0)

In [17]:
#Carregar Dada Referencia Alt Ter (Masies)

masies = loadDataAltTer("L08116-72-00002", altTer)

#Carregar Dada Referencia Baix Ter (Masies)

colomers = loadDataBaixTer("L17055-72-00002", baixTer)


In [18]:
#Carreguem Dada Sant Joan per fer Probes

santJoan = loadDataAltTer("L17167-72-00001", altTer)

## Linear Regression Test

In [30]:
#Alineem la dada de santJoan amb les dades de masies
dataX = alignSeries(santJoan, 180)[0:-6].reshape((-1,1))
dataY = masies['data'].to_numpy()


#Separem en train y test
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.2)



In [31]:
#Fittejem un model
lm = linear_model.LinearRegression()

model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

In [33]:
#Plot del model
plot

array([14.59850441,  5.26266005, 15.02535086, 10.58860816, 31.28961634])

In [ ]:
fig = px.scatter(x=y_test, y=predictions)
fig.show()

In [38]:
print('Model Score: '), model.score(X_test, y_test)

Model Score: 


(None, 0.584838271526159)

In [40]:
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, predictions), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, predictions), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, predictions), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, predictions), 2)) 
print("R2 score =", round(sm.r2_score(y_test, predictions), 2))

Mean absolute error = 5.86
Mean squared error = 456.16
Median absolute error = 2.87
Explain variance score = 0.58
R2 score = 0.58
